<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>Deep Learning Para Aplicações de Inteligência Artificial com Python e C++</font>
## <font color='blue'>Estudo de Caso 2</font>
## <font color='blue'>Construindo e Treinando Um LLM a Partir do Zero</font>

In [1]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [2]:
import random
random.seed(10)

In [3]:
# Imports
import re
import math
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import *

In [4]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" 

Author: Data Science Academy



## Carregando os Dados de Texto

In [5]:
# Carrega os dados de texto
dsa_texto = open('texto.txt', 'r').read()

In [6]:
type(dsa_texto)

str

In [7]:
print(dsa_texto)

'Olá, como vai? Eu sou a Camila.\n'
'Olá, Camila, meu nome é Fernando. Muito prazer.\n'
'Prazer em conhecer você também. Como você está hoje?\n'
'Ótimo. Meu time de futebol venceu a competição.\n'
'Uau, Parabéns Fernando!\n'
'Obrigado Camila.\n'
'Vamos comer uma pizza mais tarde para celebrar?\n'
'Claro. Você recomenda algum restaurante Camila?\n'
'Sim, abriu um restaurante novo e dizem que a pizza de banana é fenomenal.\n'
'Ok. Nos encontramos no restaurante às sete da noite, pode ser?\n'
'Pode sim. Nos vemos mais tarde então.'


## Pré-Processamento dos Dados de Texto e Construção do Vocabulário

In [8]:
# Filtramos caracteres especiais: '.', ',', '?', '!'
sentences = re.sub("[.,!?\\-]", '', dsa_texto.lower()).split('\n') 

In [9]:
print(sentences)

["'olá como vai eu sou a camila\\n'", "'olá camila meu nome é fernando muito prazer\\n'", "'prazer em conhecer você também como você está hoje\\n'", "'ótimo meu time de futebol venceu a competição\\n'", "'uau parabéns fernando\\n'", "'obrigado camila\\n'", "'vamos comer uma pizza mais tarde para celebrar\\n'", "'claro você recomenda algum restaurante camila\\n'", "'sim abriu um restaurante novo e dizem que a pizza de banana é fenomenal\\n'", "'ok nos encontramos no restaurante às sete da noite pode ser\\n'", "'pode sim nos vemos mais tarde então'"]


In [10]:
# Dividimos as frases em palavras e criamos uma lista de palavras
word_list = list(set(" ".join(sentences).split()))

In [11]:
print(word_list)

['novo', "fernando\\n'", "'ótimo", 'comer', 'abriu', 'sou', "'ok", 'noite', 'em', 'pode', "'vamos", 'eu', 'é', 'às', "'claro", 'sim', "celebrar\\n'", 'pizza', 'está', 'da', 'nome', 'tarde', 'um', 'vemos', 'de', "fenomenal\\n'", 'uma', "'sim", 'vai', 'você', 'no', "ser\\n'", "prazer\\n'", "competição\\n'", "hoje\\n'", 'meu', 'algum', "então'", "camila\\n'", 'muito', 'banana', "'prazer", 'futebol', 'sete', 'time', 'mais', 'também', 'fernando', 'restaurante', 'nos', 'como', 'recomenda', 'parabéns', "'uau", 'dizem', 'e', 'para', 'conhecer', "'olá", 'a', 'camila', 'que', "'pode", 'venceu', "'obrigado", 'encontramos']


In [12]:
# Inicializa o dicionário de palavras com os tokens especiais do BERT
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}

In [13]:
print(word_dict)

{'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}


In [14]:
# Incluímos as palavras no dicionário e criamos índices
for i, w in enumerate(word_list):
    word_dict[w] = i + 4

In [15]:
print(word_dict)

{'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3, 'novo': 4, "fernando\\n'": 5, "'ótimo": 6, 'comer': 7, 'abriu': 8, 'sou': 9, "'ok": 10, 'noite': 11, 'em': 12, 'pode': 13, "'vamos": 14, 'eu': 15, 'é': 16, 'às': 17, "'claro": 18, 'sim': 19, "celebrar\\n'": 20, 'pizza': 21, 'está': 22, 'da': 23, 'nome': 24, 'tarde': 25, 'um': 26, 'vemos': 27, 'de': 28, "fenomenal\\n'": 29, 'uma': 30, "'sim": 31, 'vai': 32, 'você': 33, 'no': 34, "ser\\n'": 35, "prazer\\n'": 36, "competição\\n'": 37, "hoje\\n'": 38, 'meu': 39, 'algum': 40, "então'": 41, "camila\\n'": 42, 'muito': 43, 'banana': 44, "'prazer": 45, 'futebol': 46, 'sete': 47, 'time': 48, 'mais': 49, 'também': 50, 'fernando': 51, 'restaurante': 52, 'nos': 53, 'como': 54, 'recomenda': 55, 'parabéns': 56, "'uau": 57, 'dizem': 58, 'e': 59, 'para': 60, 'conhecer': 61, "'olá": 62, 'a': 63, 'camila': 64, 'que': 65, "'pode": 66, 'venceu': 67, "'obrigado": 68, 'encontramos': 69}


In [16]:
# Invertemos a ordem e colocamos os índices como chave e as palavras como valor no dicionário
number_dict = {i: w for i, w in enumerate(word_dict)}

In [17]:
number_dict

{0: '[PAD]',
 1: '[CLS]',
 2: '[SEP]',
 3: '[MASK]',
 4: 'novo',
 5: "fernando\\n'",
 6: "'ótimo",
 7: 'comer',
 8: 'abriu',
 9: 'sou',
 10: "'ok",
 11: 'noite',
 12: 'em',
 13: 'pode',
 14: "'vamos",
 15: 'eu',
 16: 'é',
 17: 'às',
 18: "'claro",
 19: 'sim',
 20: "celebrar\\n'",
 21: 'pizza',
 22: 'está',
 23: 'da',
 24: 'nome',
 25: 'tarde',
 26: 'um',
 27: 'vemos',
 28: 'de',
 29: "fenomenal\\n'",
 30: 'uma',
 31: "'sim",
 32: 'vai',
 33: 'você',
 34: 'no',
 35: "ser\\n'",
 36: "prazer\\n'",
 37: "competição\\n'",
 38: "hoje\\n'",
 39: 'meu',
 40: 'algum',
 41: "então'",
 42: "camila\\n'",
 43: 'muito',
 44: 'banana',
 45: "'prazer",
 46: 'futebol',
 47: 'sete',
 48: 'time',
 49: 'mais',
 50: 'também',
 51: 'fernando',
 52: 'restaurante',
 53: 'nos',
 54: 'como',
 55: 'recomenda',
 56: 'parabéns',
 57: "'uau",
 58: 'dizem',
 59: 'e',
 60: 'para',
 61: 'conhecer',
 62: "'olá",
 63: 'a',
 64: 'camila',
 65: 'que',
 66: "'pode",
 67: 'venceu',
 68: "'obrigado",
 69: 'encontramos'}

In [18]:
# Tamanho do vocabulário
vocab_size = len(word_dict)
print(vocab_size)

70


In [19]:
# Criamos uma lista para os tokens
token_list = list()

In [20]:
# Loop pelas sentenças para criar a lista de tokens
for sentence in sentences:
    arr = [word_dict[s] for s in sentence.split()]
    token_list.append(arr)

In [21]:
token_list

[[62, 54, 32, 15, 9, 63, 42],
 [62, 64, 39, 24, 16, 51, 43, 36],
 [45, 12, 61, 33, 50, 54, 33, 22, 38],
 [6, 39, 48, 28, 46, 67, 63, 37],
 [57, 56, 5],
 [68, 42],
 [14, 7, 30, 21, 49, 25, 60, 20],
 [18, 33, 55, 40, 52, 42],
 [31, 8, 26, 52, 4, 59, 58, 65, 63, 21, 28, 44, 16, 29],
 [10, 53, 69, 34, 52, 17, 47, 23, 11, 13, 35],
 [66, 19, 53, 27, 49, 25, 41]]

In [22]:
# Primeira frase
dsa_texto[0:29]

"'Olá, como vai? Eu sou a Cami"

In [23]:
# Primeira frase no formato de token (o que será usado para treinar o modelo BERT)
token_list[0]

[62, 54, 32, 15, 9, 63, 42]

## Definição dos Hiperparâmetros

In [24]:
# Hiperparâmetros
batch_size = 6
n_segments = 2
dropout = 0.2

# Comprimento máximo
maxlen = 100 

# Número máximo de tokens que serão previstos
max_pred = 7

# Número de camadas 
n_layers = 6 

# Número de cabeças no multi-head attention
n_heads = 12

# Tamanho da embedding
d_model = 768

# Tamanho da dimensão feedforward: 4 * d_model
d_ff = d_model * 4

# Dimensão de K(=Q)V
d_k = d_v = 64 

# Epochs
NUM_EPOCHS = 50

## Criação dos Batches de Dados e Aplicação dos Tokens Especiais

A função dsa_make_batch() abaixo cria lotes (batches) de dados para o treinamento do modelo BERT. Ela é responsável por gerar a entrada correta necessária para o treinamento do BERT, que inclui os tokens de entrada, os tokens mascarados, as posições dos tokens mascarados, os IDs de segmentos e um rótulo indicando se a segunda sentença segue imediatamente a primeira. Vamos descrever cada uma das partes da função e usar imagens para facilitar a compreensão.

**Inicialização**: A função começa inicializando um lote vazio e contadores para sentenças positivas e negativas. Sentenças positivas são pares de sentenças onde a segunda sentença segue imediatamente a primeira, enquanto as negativas são pares onde isso não ocorre. O lote deve ser equilibrado entre sentenças positivas e negativas.

**Geração de pares de sentenças**: Para cada instância no lote, a função seleciona aleatoriamente duas sentenças do conjunto de dados. Cada sentença é então convertida em uma lista de IDs de tokens e os tokens especiais [CLS] e [SEP] são adicionados nos lugares apropriados.

**Segment IDs**: Para cada par de sentenças, a função gera IDs de segmentos, que são 0 para tokens na primeira sentença e 1 para tokens na segunda sentença.

**Masked Language Model (MLM)**: A função então seleciona aleatoriamente 15% dos tokens para mascarar para a tarefa de MLM, garantindo que os tokens [CLS] e [SEP] não sejam mascarados. Esses tokens são substituídos pelo token [MASK], por um token aleatório ou permanecem inalterados, dependendo de um sorteio aleatório.

![DSA](imagens/dsa_bert1.png)

**Padding**: A função adiciona padding aos IDs de entrada, aos IDs de segmento, aos tokens mascarados e às posições mascaradas para garantir que todas as listas tenham o mesmo comprimento.

**Next Sentence Prediction**: Por último, a função verifica se a segunda sentença segue imediatamente a primeira. Se sim, ela adiciona um rótulo True à instância e incrementa o contador de positivos. Se não, ela adiciona um rótulo False e incrementa o contador de negativos.

![DSA](imagens/dsa_bert2.png)

Esta função continua gerando instâncias até que o lote esteja cheio e contenha uma quantidade igual de instâncias positivas e negativas. Então, o lote é retornado.

Note que esta função é apenas um exemplo de como os dados podem ser preparados para o treinamento do BERT. Dependendo do conjunto de dados e da tarefa específica, pode ser necessário ajustar esta função.

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

A principal inovação técnica do BERT é aplicar o treinamento bidirecional do Transformer, um modelo de atenção popular, à modelagem de linguagem. Isso contrasta com os esforços anteriores que analisavam uma sequência de texto da esquerda para a direita ou um treinamento combinado da esquerda para a direita e da direita para a esquerda. Os resultados do artigo mostram que um modelo de linguagem que é treinado bidirecionalmente pode ter um senso mais profundo de contexto e fluxo de linguagem do que modelos de linguagem de direção única. 

No artigo, os pesquisadores detalham uma nova técnica chamada Masked LM (MLM), que permite o treinamento bidirecional em modelos nos quais antes era impossível. 

Link do artigo do BERT: https://arxiv.org/abs/1810.04805

In [25]:
# Define a função para criar lotes de dados
def dsa_make_batch():
    
    # Inicializa o lote como uma lista vazia
    batch = []
    
    # Inicializa contadores para exemplos positivos e negativos
    positive = negative = 0
    
    # Continua até que a metade do lote seja de exemplos positivos e a outra metade de exemplos negativos
    while positive != batch_size/2 or negative != batch_size/2:
        
        # Escolhe índices aleatórios para duas sentenças
        tokens_a_index, tokens_b_index = randrange(len(sentences)), randrange(len(sentences))
        
        # Recupera os tokens correspondentes aos índices
        tokens_a, tokens_b = token_list[tokens_a_index], token_list[tokens_b_index]
        
        # Prepara os ids de entrada adicionando tokens especiais [CLS] e [SEP]
        input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']]
        
        # Define os segment ids para diferenciar as duas sentenças
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)
        
        # Calcula o número de previsões a serem feitas (15% dos tokens)
        n_pred =  min(max_pred, max(1, int(round(len(input_ids) * 0.15)))) 
        
        # Identifica as posições candidatas para mascaramento que não sejam [CLS] ou [SEP]
        cand_maked_pos = [i for i, token in enumerate(input_ids) if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]
        
        # Embaralha as posições candidatas
        shuffle(cand_maked_pos)
        
        # Inicializa listas para tokens mascarados e suas posições
        masked_tokens, masked_pos = [], []
        
        # Mascara tokens até atingir o número de previsões desejado
        for pos in cand_maked_pos[:n_pred]:
            masked_pos.append(pos)
            masked_tokens.append(input_ids[pos])
            
            # Máscara aleatória
            if random() < 0.8:  
                input_ids[pos] = word_dict['[MASK]'] 
            
            # Substitui por outro token 10% das vezes (20% do tempo restante)
            elif random() < 0.5:  
                index = randint(0, vocab_size - 1) 
                input_ids[pos] = word_dict[number_dict[index]] 
        
        # Adiciona zero padding aos ids de entrada e segment ids para atingir o comprimento máximo
        n_pad = maxlen - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)
        
        # Adiciona zero padding aos tokens mascarados e suas posições se necessário
        if max_pred > n_pred:
            n_pad = max_pred - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)
        
        # Adiciona ao lote como um exemplo positivo se as sentenças forem consecutivas
        if tokens_a_index + 1 == tokens_b_index and positive < batch_size / 2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) 
            positive += 1
        
        # Adiciona ao lote como um exemplo negativo se as sentenças não forem consecutivas
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size / 2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) 
            negative += 1
    
    # Retorna o lote completo
    return batch

In [26]:
# Função para o padding
def dsa_get_attn_pad_masked(seq_q, seq_k):
    
    batch_size, len_q = seq_q.size()
    
    batch_size, len_k = seq_k.size()
    
    pad_attn_masked = seq_k.data.eq(0).unsqueeze(1)
    
    return pad_attn_masked.expand(batch_size, len_q, len_k)

A função acima cria uma máscara de atenção para tokens de padding em uma sequência.

**Entradas**: A função aceita duas sequências, seq_q e seq_k. Estas são tipicamente a sequência de consulta (query) e a sequência chave (key) em uma operação de atenção.

**Extração de tamanho**: A função extrai o tamanho do lote (batch_size) e os comprimentos das sequências (len_q e len_k) a partir das dimensões das sequências de entrada.

**Criação da máscara**: A máscara de atenção é criada verificando quais elementos em seq_k são iguais a zero (o que indica um token de padding). Isso produz uma matriz booleana do mesmo tamanho que seq_k, onde True indica um token de padding e False indica um token real.

**Adição de uma dimensão**: A dimensão é adicionada à máscara usando o método unsqueeze(1), que adiciona uma dimensão extra no índice 1. Isso é necessário porque a máscara de atenção deve ter a mesma dimensão que as matrizes de atenção no Transformer.

**Expansão da máscara**: Finalmente, a máscara é expandida para ter o mesmo tamanho que a matriz de atenção, que tem dimensões (batch_size, len_q, len_k). A máscara expandida é retornada pela função.

Em resumo, a função cria uma máscara que pode ser usada para impedir que o modelo preste atenção aos tokens de padding quando calcula a atenção. Tokens de padding são usados para preencher sequências para que todas tenham o mesmo comprimento, mas eles não carregam nenhuma informação útil, por isso é importante garantir que o modelo os ignore.

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

In [27]:
# Cria um batch
batch = dsa_make_batch()

In [28]:
# Extrai os elementos do batch
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

In [29]:
# Ids das sentenças
input_ids

tensor([[ 1, 10, 53, 69, 34, 52,  3, 47, 23, 11, 13, 35,  2,  3, 54, 32, 15,  9,
         63, 42,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 66, 19, 53, 27, 49, 25,  3,  2, 57, 56,  3,  2,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 62, 54, 32, 15,  9, 63, 42,  2,  6, 39, 48, 28,  3,  3, 63, 37,  2,
         

In [30]:
# Ids de entrada da primeira sentença
input_ids[0]

tensor([ 1, 10, 53, 69, 34, 52,  3, 47, 23, 11, 13, 35,  2,  3, 54, 32, 15,  9,
        63, 42,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [31]:
segment_ids[0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])

In [32]:
masked_tokens[0]

tensor([15, 62, 17,  0,  0,  0,  0])

In [33]:
masked_pos[0]

tensor([16, 13,  6,  0,  0,  0,  0])

In [34]:
isNext[0]

tensor(0)

In [35]:
# Aplica a função de padding
dsa_get_attn_pad_masked(input_ids, input_ids)[0][0], input_ids[0]

(tensor([False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True]),
 tensor([ 1, 10, 53, 69, 34, 52,  3, 47, 23, 11, 13, 35,  2,  3, 54, 32, 15,  9,
         63, 42,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  

## Construção do Modelo

A imagem abaixo é uma descrição de alto nível do codificador Transformer. A entrada é uma sequência de tokens, que são primeiro incorporados em vetores e depois processados na rede neural. A saída é uma sequência de vetores de tamanho H, em que cada vetor corresponde a um token de entrada com o mesmo índice.

Em termos técnicos, a previsão das palavras de saída requer:

- 1- Adicionar uma camada de classificação na parte superior da saída do codificador.
- 2- Multiplicar os vetores de saída pela matriz embedding, transformando-os na dimensão do vocabulário.
- 3- Calcular a probabilidade de cada palavra no vocabulário com softmax.

A função de perda no modelo BERT leva em consideração apenas a previsão dos valores mascarados e ignora a previsão das palavras não mascaradas. Como consequência, o modelo converge mais lentamente do que os modelos direcionais, uma característica que é compensada por sua maior percepção do contexto.

![DSA](imagens/dsa_bert3.png)

No processo de treinamento BERT, o modelo recebe pares de sentenças como entrada e aprende a prever se a segunda sentença do par é a sentença subsequente no documento original. Durante o treinamento, 50% das entradas são um par em que a segunda sentença é a sentença subsequente no documento original, enquanto nos outros 50% uma sentença aleatória do corpus é escolhida como a segunda sentença. 

Para ajudar o modelo a distinguir entre as duas sentenças em treinamento, a entrada é processada da seguinte maneira antes de entrar no modelo:

- 1- Um token [CLS] é inserido no início da primeira frase e um token [SEP] é inserido no final de cada frase.
- 2- Uma embedding de frase indicando a Sentença A ou a Sentença B é adicionada a cada token. As embeddings de sentença são semelhantes em conceito às embeddings de token com um vocabulário de 2.
- 3- Uma embedding posicional é adicionada a cada token para indicar sua posição na sequência. O conceito e a implementação da embedding posicional são apresentados no artigo Transformer.

De fato a embedding usada para treinar o modelo é uma combinação de várias embeddings.

![DSA](imagens/dsa_bert4.png)
<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

In [36]:
# Função de ativação GeLu
def gelu(x):
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

### 1- Módulo Embedding

A classe de Embedding abaixo faz parte da arquitetura BERT. Componentes individuais da Classe:

Inicialização (def init(self)): O construtor da classe inicializa os componentes necessários para as embeddings.

self.tok_embed: Esta é a camada de embedding de token que mapeia cada token para um vetor de dimensão d_model.

self.pos_embed: Esta é a camada de embedding de posição que mapeia a posição de um token dentro de uma sequência para um vetor de dimensão d_model.

self.seg_embed: Esta é a camada de embedding de segmento que mapeia o tipo de token (0 para a primeira sentença e 1 para a segunda sentença) para um vetor de dimensão d_model.

self.norm: Este é o componente de normalização da camada que é usado para normalizar os vetores de embedding.

Método Forward (def forward(self, x, seg)): O método forward é onde a embedding real acontece.

- Primeiro, ele calcula a posição de cada token na sequência.
- Em seguida, ele cria uma matriz de posições da mesma forma que a entrada x usando pos.unsqueeze(0).expand_as(x).
- Depois, ele calcula a embedding total como a soma das embeddings de token, posição e segmento.
- Finalmente, ele normaliza a embedding usando a camada de normalização e retorna o resultado.

A combinação dessas três embeddings permite ao BERT levar em consideração tanto o significado individual dos tokens quanto a ordem em que aparecem na sequência, bem como se o token pertence à primeira ou à segunda sentença. Isso torna a embedding do BERT muito poderosa e flexível.

In [37]:
# Classe Embedding
class Embedding(nn.Module):
    
    # Método construtor
    def __init__(self):
        
        super(Embedding, self).__init__()
        
        # Token embedding
        self.tok_embed = nn.Embedding(vocab_size, d_model)  
        
        # Position embedding
        self.pos_embed = nn.Embedding(maxlen, d_model)  
        
        # Segment (tipo de token) embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  
        
        # Normalização de camada
        self.norm = nn.LayerNorm(d_model)

    # Método Forward
    def forward(self, x, seg):
        
        seq_len = x.size(1)
        
        pos = torch.arange(seq_len, dtype = torch.long)
        
        # (seq_len,) -> (batch_size, seq_len)
        pos = pos.unsqueeze(0).expand_as(x)  
        
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        
        return self.norm(embedding)

### 2- Módulo Scaled Dot Product Attention

Abaixo está a implementação do mecanismo de Atenção por Produto Escalar Normalizado (Scaled Dot-Product Attention), que é uma parte chave do modelo Transformer, utilizado no BERT e em outros modelos de processamento de linguagem natural.

Aqui está uma explicação linha a linha do método forward:

**Pontuações (Scores)**: O produto escalar de Q (matriz de consulta) e K (matriz chave) é calculado para determinar a pontuação para cada par de chave-consulta. Essas pontuações determinam o quanto cada elemento da sequência de entrada deve ser atendido na produção da representação de saída para um determinado elemento. A pontuação é então escalada pela raiz quadrada da dimensão das chaves (d_k) para evitar que os valores de produto escalar se tornem muito grandes em ambientes de alta dimensão.

**Máscara de Atenção**: A máscara de atenção é aplicada às pontuações ao preencher os locais onde a máscara tem valor 1 com um número muito grande negativo (-1e9). Isso garante que esses locais recebam um peso próximo de zero quando a softmax for aplicada.

**Softmax**: A função softmax é aplicada ao último eixo das pontuações para obter os pesos de atenção. Isso garante que todos os pesos são positivos e somam 1, então eles podem ser interpretados como probabilidades.

**Contexto**: Os pesos de atenção são então multiplicados pela matriz de valores V (value) para obter a saída do mecanismo de atenção. Cada valor é ponderado pela quantidade que deveríamos "atender" a esse valor, conforme determinado pelos pesos de atenção.

O método retorna o contexto (a saída ponderada) e a matriz de atenção.

No modelo Transformer, a Atenção por Produto Escalar Normalizado é usada várias vezes em cada camada, permitindo que o modelo dê atenção a diferentes partes da entrada ao produzir cada elemento da saída. Isso permite que o Transformer lide efetivamente com as dependências de longo alcance entre as palavras nas sequências de entrada.

In [38]:
# Define a classe para realizar a atenção por produto escalar normalizado
class ScaledDotProductAttention(nn.Module):
    
    # Método de inicialização da classe
    def __init__(self):
        
        # Inicializa a classe base
        super(ScaledDotProductAttention, self).__init__()

    # Método forward para definir a passagem para frente dos dados
    def forward(self, Q, K, V, attn_mask):
        
        # Calcula os scores de atenção como o produto de Q e K, e normaliza pelo tamanho da chave
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)
        
        # Aplica a máscara de atenção para evitar atenção a certos tokens
        scores.masked_fill_(attn_mask, -1e9)
        
        # Aplica a softmax para obter pesos de atenção normalizados
        attn = nn.Softmax(dim = -1)(scores)
        
        # Multiplica os pesos de atenção por V para obter o contexto
        context = torch.matmul(attn, V)
        
        # Retorna o contexto e os pesos de atenção
        return context, attn

### 3- Módulo Multi-Head Attention

Abaixo está a implementação de Atenção Multi-Cabeças (Multi-Head Attention), que é um componente chave da arquitetura Transformer, usada em modelos como BERT. A ideia da atenção multi-cabeças é aplicar a atenção do produto escalar normalizado várias vezes em paralelo, cada uma com diferentes pesos aprendidos. Isso permite ao modelo focar em diferentes posições e capturar vários tipos de informação.

Vamos analisar linha a linha o método forward:

**Inicialização**: residual e batch_size são inicializados com Q e o tamanho do primeiro eixo de Q, respectivamente. O residual será utilizado mais tarde para o caminho de conexão residual.

**Transformações Lineares**: Aplicamos transformações lineares aos dados de entrada (Q, K e V) usando pesos diferentes. Essas transformações geram múltiplas "cabeças" de atenção.

**Remodelagem**: As saídas dessas transformações lineares são então reformuladas e transpostas para terem a forma apropriada para a atenção de produto escalar normalizado.

**Máscara de Atenção**: A máscara de atenção é ajustada para corresponder ao formato das cabeças de atenção.

**Atenção de Produto Escalar Normalizado**: A atenção de produto escalar normalizado é então aplicada a cada uma das cabeças de atenção.

**Remodelagem do Contexto**: A saída (contexto) de cada cabeça de atenção é então reformulada e concatenada.

**Transformação Linear e Normalização**: Uma transformação linear é aplicada ao contexto concatenado, seguida de uma normalização de camada.

**Conexão Residual**: O resultado final é obtido somando a saída da normalização de camada ao caminho de conexão residual (entrada original Q).

Finalmente, a função retorna a saída normalizada e a matriz de atenção. A atenção multi-cabeças permite que o modelo considere informações de diferentes partes da sequência de entrada, em diferentes subespaços de representação, ao mesmo tempo, o que melhora a capacidade do modelo de capturar várias características do texto.

In [39]:
# Define a classe para realizar atenção multi-cabeça
class MultiHeadAttention(nn.Module):
    
    def __init__(self) -> None:
        
        # Inicializa a classe base
        super(MultiHeadAttention, self).__init__()
        
        # Define a matriz de pesos para as consultas Q
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        
        # Define a matriz de pesos para as chaves K
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        
        # Define a matriz de pesos para os valores V
        self.W_V = nn.Linear(d_model, d_v * n_heads)

    # Método forward para definir a passagem para frente dos dados
    def forward(self, Q, K, V, attn_mask):
        
        # Salva a entrada Q para usar no residual e obtém o tamanho do batch
        residual, batch_size = Q, Q.size(0)
        
        # Processa Q através de W_Q e organiza o resultado para ter [n_heads] na segunda dimensão
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)
        
        # Processa K através de W_K e organiza o resultado para ter [n_heads] na segunda dimensão
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)
        
        # Processa V através de W_V e organiza o resultado para ter [n_heads] na segunda dimensão
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)
        
        # Adapta attn_mask para ser compatível com as dimensões de q_s, k_s, v_s
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)
        
        # Calcula a atenção escalonada do produto ponto e o contexto para cada cabeça de atenção
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        
        # Reorganiza o contexto para combinar as cabeças de atenção e volta para o formato original
        context = context.transpose(1,2).contiguous().view(batch_size, -1, n_heads * d_v)
        
        # Aplica uma transformação linear ao contexto combinado
        output = nn.Linear(n_heads * d_v, d_model)(context)
        
        # Normaliza a camada de saída e adiciona o residual
        return nn.LayerNorm(d_model)(output + residual), attn

In [40]:
# Cria o objeto Embedding
emb = Embedding()

In [41]:
# Gera as Embeddings
embeds = emb(input_ids, segment_ids)

In [42]:
# Gera a máscara de atenção
attenM = dsa_get_attn_pad_masked(input_ids, input_ids)

In [43]:
# Gera o MultiHeadAttention
MHA = MultiHeadAttention()(embeds, embeds, embeds, attenM)

In [44]:
# Saída
output, A = MHA

In [45]:
A[0][0]

tensor([[0.0404, 0.0240, 0.0439,  ..., 0.0000, 0.0000, 0.0000],
        [0.0359, 0.0259, 0.0623,  ..., 0.0000, 0.0000, 0.0000],
        [0.0419, 0.0280, 0.0538,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0354, 0.0390, 0.0521,  ..., 0.0000, 0.0000, 0.0000],
        [0.0481, 0.0355, 0.0540,  ..., 0.0000, 0.0000, 0.0000],
        [0.0324, 0.0418, 0.0460,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<SelectBackward0>)

### 4- Módulo Feedforward Posicional

Esta é a implementação da Rede Feedforward Posicional (PoswiseFeedForward), que é um componente da arquitetura Transformer, utilizada em modelos como o BERT.

A Rede Feedforward Posicional é composta por duas camadas lineares com uma ativação GELU (Gaussian Error Linear Unit) entre elas.

Aqui está a explicação detalhada do método forward:

**Primeira Camada Linear (self.fc1)**: A entrada x é passada por uma camada linear (também chamada de camada totalmente conectada). Essa camada tem uma transformação linear com d_model entradas e d_ff saídas, onde d_model é a dimensão do espaço de incorporação e d_ff é a dimensão da camada oculta da rede feed-forward. Isso permite ao modelo aprender representações não-lineares.

**Ativação GELU**: Em seguida, a ativação GELU é aplicada. A função GELU permite ao modelo aprender transformações mais complexas e não lineares. Ela ajuda a lidar com o problema do desaparecimento do gradiente, permitindo que mais informações passem através da rede.

**Segunda Camada Linear (self.fc2)**: Por fim, a saída da ativação GELU é passada por uma segunda camada linear, que transforma a saída de volta para a dimensão original d_model. Isso é feito para que a saída desta rede feedforward possa ser somada à entrada original (conexão residual) no Transformer.

O retorno da função é, portanto, a saída dessa segunda camada linear, que passou pela transformação da primeira camada linear, ativação GELU, e segunda camada linear.

As redes feed-forward posicionais são uma parte importante dos modelos Transformer, permitindo-lhes aprender representações mais complexas e fazer transformações não-lineares dos dados de entrada.

In [46]:
# Define a classe para a rede Feed Forward Posicional
class PoswiseFeedForward(nn.Module):
    
    def __init__(self) -> None:
        
        # Inicializa a classe base
        super(PoswiseFeedForward, self).__init__()
        
        # Primeira camada linear que aumenta a dimensão dos dados de d_model para d_ff
        self.fc1 = nn.Linear(d_model, d_ff)
        
        # Segunda camada linear que reduz a dimensão de volta de d_ff para d_model
        self.fc2 = nn.Linear(d_ff, d_model)

    # Método forward para definir a passagem para frente dos dados
    def forward(self, x):
        
        # Aplica a primeira transformação linear, seguida pela função de ativação GELU 
        # e então a segunda transformação linear
        return self.fc2(gelu(self.fc1(x)))

### 5- Módulo Encoder Layer

Esta classe define uma Camada de Codificador (EncoderLayer), que é um componente da arquitetura Transformer e também é usado em modelos como BERT. Cada camada de codificador no Transformer contém duas subcamadas: uma camada de Atenção Multi-Cabeças e uma Rede Feed-Forward Posicional.

Aqui está a explicação detalhada do método forward:

**Atenção Multi-Cabeças (self.enc_self_attn)**: A entrada enc_inputs passa por uma camada de Atenção Multi-Cabeças, que é usada para que cada palavra na entrada tenha atenção direcionada a todas as outras palavras. Essa camada também recebe uma máscara (enc_self_attn_mask), que é usada para evitar que o modelo preste atenção a certas palavras (como as de preenchimento). A saída da Atenção Multi-Cabeças é outra sequência de representações vetoriais, com a mesma dimensão da entrada. A matriz de atenção que mostra como cada palavra se atentou a todas as outras também é retornada.

**Rede Feed-Forward Posicional (self.pos_ffn)**: A saída da camada de Atenção Multi-Cabeças passa então por uma Rede Feed-Forward Posicional. Esta é uma rede neural simples que opera independentemente em cada posição da sequência (ou seja, a mesma rede é aplicada a cada posição). Isso permite ao modelo aprender representações mais complexas e realizar transformações não-lineares dos dados.

A função retorna a saída desta camada de codificador, que é a saída da Rede Feed-Forward Posicional, junto com a matriz de atenção. Portanto, a entrada e a saída desta camada do codificador têm a mesma dimensão, o que permite que várias dessas camadas de codificador sejam empilhadas para formar o codificador completo do Transformer.

In [47]:
# Define a classe para a camada do codificador
class EncoderLayer(nn.Module):
    
    def __init__(self) -> None:
        
        # Inicializa a classe base
        super(EncoderLayer, self).__init__()
        
        # Instancia a atenção multi-cabeças para a auto-atenção do codificador
        self.enc_self_attn = MultiHeadAttention()
        
        # Instancia a rede Feed Forward Posicional para usar após a auto-atenção
        self.pos_ffn = PoswiseFeedForward()

    # Método forward para definir a passagem para frente dos dados
    def forward(self, enc_inputs, enc_self_attn_mask):
        
        # Aplica auto-atenção aos dados de entrada
        enc_inputs, atnn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask)
        
        # Após a auto-atenção, passa o resultado pela rede Feed Forward Posicional
        enc_inputs = self.pos_ffn(enc_inputs)
        
        # Retorna a saída do codificador e os pesos de atenção
        return enc_inputs, atnn

### 6- Arquitetura Final do LLM (Modelo BERT)

Esta classe define o modelo BERT (Bidirectional Encoder Representations from Transformers), um modelo de linguagem de última geração que usa transformers e atenção bidirecional para entender a semântica das palavras dentro de um contexto.

Vamos analisar em detalhes o método forward:

Embedding (self.embedding): Transforma as entradas (input_ids e segment_ids) em vetores densos (embeddings).

Máscara de Atenção (get_attn_pad_masked): Gera uma máscara de atenção para ignorar os tokens de preenchimento (pad) nas entradas.

Camadas de Codificação (self.layers): Passa a saída do embedding e a máscara de atenção através de várias camadas do codificador. Cada camada de codificador é composta por uma camada de atenção multi-cabeças e uma rede feed-forward posicional.

Pooling (self.activ1(self.fc(output[:, 0]))): Aplica uma camada totalmente conectada e uma ativação tangente hiperbólica à primeira posição (o token de classificação) de cada sequência na saída do codificador. Isso resulta em um vetor de representação de sequência.

Classificador (self.classifier): Uma camada totalmente conectada que gera os logits para a tarefa de classificação de próxima sentença.

Extração de Tokens Mascarados (torch.gather(output, 1, masked_pos)): Selecione os vetores de saída correspondentes aos tokens mascarados.

Transformação dos Tokens Mascarados (self.norm(self.activ2(self.linear(h_masked)))): Aplica uma transformação linear, uma ativação GELU e normalização à saída dos tokens mascarados.

Decoder (self.decoder): Uma camada linear que gera os logits para a tarefa de modelagem de linguagem mascarada. Usa os mesmos pesos que a camada de embedding de tokens para a consistência no espaço de representação. Esta função decoder é usada somente para gerar os logits finais e não é usada no processo de aprendizado do modelo.

O método retorna os logits para a tarefa de modelagem de linguagem mascarada e a tarefa de classificação de próxima sentença. Esses logits podem então ser usados para calcular as perdas para ambas as tarefas durante o treinamento.

In [48]:
# Modelo BERT
class BERT(nn.Module):
    
    def __init__(self) -> None:
        
        super(BERT, self).__init__()
        
        self.embedding = Embedding()
        
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        
        self.fc = nn.Linear(d_model, d_model)
        
        self.activ1 = nn.Tanh()
        
        self.linear = nn.Linear(d_model, d_model)
        
        self.activ2 = gelu
        
        self.norm = nn.LayerNorm(d_model)
        
        self.classifier = nn.Linear(d_model, 2)
        
        embed_weight = self.embedding.tok_embed.weight
        
        n_vocab, n_dim = embed_weight.size()
        
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        
        self.decoder.weight = embed_weight
        
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

    def forward(self, input_ids, segment_ids, masked_pos):
        
        output = self.embedding(input_ids, segment_ids)
        
        enc_self_attn_mask = dsa_get_attn_pad_masked(input_ids, input_ids)
        
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        
        h_pooled = self.activ1(self.fc(output[:, 0]))
        
        logits_clsf = self.classifier(h_pooled)
        
        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1))
        
        h_masked = torch.gather(output, 1, masked_pos)
        
        h_masked = self.norm(self.activ2(self.linear(h_masked)))
        
        logits_lm = self.decoder(h_masked) + self.decoder_bias
        
        return logits_lm, logits_clsf

## Treinamento e Avaliação do LLM

In [49]:
# Cria o modelo
modelo_dsa = BERT()

In [50]:
# Função de erro
criterion = nn.CrossEntropyLoss()

In [51]:
# Otimizador
optimizer = optim.Adam(modelo_dsa.parameters(), lr = 0.001)

In [52]:
batch = dsa_make_batch()

In [53]:
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

Abaixo está o ciclo típico de treinamento de uma época em um modelo de aprendizado de máquina. Vamos quebrá-lo em etapas:

**optimizer.zero_grad()**: Zera os gradientes de todas as variáveis otimizadas. Isso é feito porque os gradientes no PyTorch são acumulados, ou seja, cada vez que chamamos .backward(), os gradientes são somados em vez de substituídos. Então, precisamos limpar esses gradientes acumulados antes de cada passo de otimização.

**logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)**: Alimenta os dados de entrada no modelo e obtém a saída do modelo. A saída é composta de logits_lm e logits_clsf, que são os resultados brutos não normalizados para a tarefa de modelagem de linguagem e a tarefa de classificação, respectivamente.

**loss_lm = criterion(logits_lm.transpose(1,2), masked_tokens)**: Calcula a perda da tarefa de modelagem de linguagem mascarada. criterion é a função de perda, logits_lm.transpose(1,2) são as previsões do modelo e masked_tokens são os alvos verdadeiros.

**loss_lm = (loss_lm.float()).mean()**: Converte a perda em um tipo de dados de ponto flutuante (se ainda não for) e, em seguida, calcula a média da perda.

**loss_clsf = criterion(logits_clsf, isNext)**: Calcula a perda da tarefa de classificação da próxima frase.

**loss = loss_lm + loss_clsf**: Combina as duas perdas em uma única perda escalar.

**loss.backward()**: Calcula os gradientes de todas as variáveis otimizadas. Os gradientes são computados com respeito à perda.

**optimizer.step()**: Atualiza os parâmetros do modelo usando os gradientes calculados.

Essas etapas são repetidas para cada época de treinamento. Cada época é um ciclo completo através do conjunto de treinamento. Portanto, se NUM_EPOCHS é 10, então o processo de treinamento completo é executado 10 vezes.

In [54]:
%%time

# Inicia o loop de treino para um número definido de épocas
for epoch in range(NUM_EPOCHS):
    
    # Zera os gradientes do otimizador para evitar acumulação de gradientes de épocas anteriores
    optimizer.zero_grad()
    
    # Passa os dados de entrada pelo modelo e obtém os logits para mascaramento de linguagem 
    # e classificação de próxima sentença
    logits_lm, logits_clsf = modelo_dsa(input_ids, segment_ids, masked_pos)
    
    # Calcula a perda para a tarefa de mascaramento de linguagem comparando os logits previstos 
    # com os tokens reais
    loss_lm = criterion(logits_lm.transpose(1,2), masked_tokens)
    
    # Calcula a média da perda para normalizar
    loss_lm = (loss_lm.float()).mean()
    
    # Calcula a perda para a tarefa de classificação de próxima sentença
    loss_clsf = criterion(logits_clsf, isNext)
    
    # Soma as perdas das duas tarefas para obter a perda total
    loss = loss_lm + loss_clsf
    
    # Exibe a época atual e a perda total
    print(f'Epoch: {epoch + 1} | Loss {loss:.4f}')
    
    # Realiza o backpropagation para calcular os gradientes
    loss.backward()
    
    # Atualiza os parâmetros do modelo com base nos gradientes calculados
    optimizer.step()

Epoch: 1 | Loss 81.6977
Epoch: 2 | Loss 89.3353
Epoch: 3 | Loss 379.5759
Epoch: 4 | Loss 46.9996
Epoch: 5 | Loss 49.3941
Epoch: 6 | Loss 19.2554
Epoch: 7 | Loss 44.1877
Epoch: 8 | Loss 51.8902
Epoch: 9 | Loss 30.9011
Epoch: 10 | Loss 13.8272
Epoch: 11 | Loss 21.3458
Epoch: 12 | Loss 22.2943
Epoch: 13 | Loss 22.1509
Epoch: 14 | Loss 23.7744
Epoch: 15 | Loss 19.7176
Epoch: 16 | Loss 19.2777
Epoch: 17 | Loss 16.9918
Epoch: 18 | Loss 19.4686
Epoch: 19 | Loss 16.4230
Epoch: 20 | Loss 15.8092
Epoch: 21 | Loss 15.7265
Epoch: 22 | Loss 15.8017
Epoch: 23 | Loss 15.2301
Epoch: 24 | Loss 16.1666
Epoch: 25 | Loss 15.7213
Epoch: 26 | Loss 16.3425
Epoch: 27 | Loss 16.2232
Epoch: 28 | Loss 15.5268
Epoch: 29 | Loss 15.1480
Epoch: 30 | Loss 14.3166
Epoch: 31 | Loss 13.4411
Epoch: 32 | Loss 14.5302
Epoch: 33 | Loss 12.7148
Epoch: 34 | Loss 12.4154
Epoch: 35 | Loss 12.2172
Epoch: 36 | Loss 12.8381
Epoch: 37 | Loss 11.8375
Epoch: 38 | Loss 12.0554
Epoch: 39 | Loss 11.1153
Epoch: 40 | Loss 10.6074
Epoch: 4

## Extraindo as Previsões do LLM Treinado

In [55]:
# Extrai o batch
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[0]))
print(dsa_texto)
print([number_dict[w.item()] for w in input_ids[0] if number_dict[w.item()] != '[PAD]'])

'Olá, como vai? Eu sou a Camila.\n'
'Olá, Camila, meu nome é Fernando. Muito prazer.\n'
'Prazer em conhecer você também. Como você está hoje?\n'
'Ótimo. Meu time de futebol venceu a competição.\n'
'Uau, Parabéns Fernando!\n'
'Obrigado Camila.\n'
'Vamos comer uma pizza mais tarde para celebrar?\n'
'Claro. Você recomenda algum restaurante Camila?\n'
'Sim, abriu um restaurante novo e dizem que a pizza de banana é fenomenal.\n'
'Ok. Nos encontramos no restaurante às sete da noite, pode ser?\n'
'Pode sim. Nos vemos mais tarde então.'
['[CLS]', "'olá", '[MASK]', 'vai', 'eu', 'sou', 'a', "camila\\n'", '[SEP]', "'ok", 'nos', 'encontramos', 'no', 'restaurante', 'às', 'sete', '[MASK]', 'noite', 'pode', "'vamos", '[SEP]']


In [56]:
# Extrai as previsões dos tokens
logits_lm, logits_clsf = modelo_dsa(input_ids, segment_ids, masked_pos)
logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
print('Lista de Masked Tokens Reais: ', [pos.item() for pos in masked_tokens[0] if pos.item() != 0])
print('Lista de Masked Tokens Previstos: ', [pos for pos in logits_lm if pos != 0])

Lista de Masked Tokens Reais:  [35, 54, 23]
Lista de Masked Tokens Previstos:  []


In [57]:
# Extrai as previsões do próximo token
logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
print('isNext (Valor Real): ', True if isNext else False)
print('isNext (Valor Previsto): ', True if logits_clsf else False)

isNext (Valor Real):  False
isNext (Valor Previsto):  False


In [58]:
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [59]:
#%watermark -v -m

In [60]:
#%watermark --iversions

# Fim